In [ ]:
### Required Package Installations (Uncomment if Needed)
# Uncomment and run these lines if the required packages are not installed
# !pip install langchain
# !pip install openai==0.28
# !pip install streamlit
# !pip install transformers

In [ ]:
### Load Environment Variables
from dotenv import load_dotenv
load_dotenv()  # This loads environment variables from a .env file

True

In [ ]:
### Retrieve API Key from Environment Variables
import os
HUGGINGFACE_API_KEY = os.getenv("HUGGINGFACE_API_KEY")  # Fetch API key securely

In [ ]:
### API Request to Hugging Face Inference Endpoint
import requests

# Define API details
API_URL = "https://api-inference.huggingface.co/models/meta-llama/Llama-2-7b-hf"
headers = {"Authorization": f"Bearer {HUGGINGFACE_API_KEY}"}  # Set authorization header

# Function to send input queries to the model
def query(payload):
	response = requests.post(API_URL, headers=headers, json=payload)
	return response.json() # Return response in JSON format

# Example API request
output = query({
	"inputs": "Can you please let us know more details about your ",
})

In [ ]:
output

{'error': 'Model meta-llama/Llama-2-7b-hf is currently loading',
 'estimated_time': 539.0735473632812}

In [ ]:
# Garbage collection and memory cleanup
import gc
gc.collect()
del pipe

In [ ]:
import langchain
import os
import openai
import streamlit as st


import warnings
# Suppress warnings
warnings.filterwarnings("ignore")

In [ ]:
from langchain.llms import OpenAI

In [ ]:
# OpenAI Model Setup
api_key = open('OpenAI API Key.txt').read()
os.environ['OPENAI_API_KEY'] = api_key

In [ ]:
# Initialize OpenAI model
llm = OpenAI(model_name='gpt-3.5-turbo')

In [ ]:
# Example query to OpenAI LLM
llm("How long does it take to become healthy")

"The time it takes to become healthy depends on various factors, such as an individual's starting point, current health conditions, lifestyle habits, and goals. There is no fixed time frame as health is a lifelong journey. However, making positive changes such as improving nutrition, incorporating regular exercise, getting enough sleep, managing stress, and quitting unhealthy habits can yield noticeable improvements in a few weeks or months. It is important to note that consistency and commitment are crucial in maintaining long-term health."

In [ ]:
from langchain.chat_models import ChatOpenAI

In [ ]:
# Initialize chat-based OpenAI model
chat = ChatOpenAI()

In [ ]:
from langchain.schema import HumanMessage, SystemMessage, HumanMessage

In [ ]:
system_message = """You are a helpful doctor who answers all the queries like a doctor.
Ensure to give a nutrition plan for your application and useful piece of advice as a doctor to the person.
Ensure that you are optimistic and give encouraging words to patient regarding their health. Also give possible
medication if necessary if the patient needs it to improve their condition.
Give medicines as a list without adding unnecessary information."""

In [ ]:
output = chat([SystemMessage(content=system_message)])

In [ ]:
from langchain.cache import InMemoryCache
langchain.llm_cache = InMemoryCache()

In [ ]:
human_message = """I'm suffering from severe fever and also cold. What are the precautions I can take to reduce cold and fever?
Suggest some tablets I could take for this."""

output = chat([SystemMessage(content=system_message),
               HumanMessage(content=human_message)])

In [ ]:
print(output.content)

I'm sorry to hear that you're feeling unwell. It's important to take some precautions to reduce your symptoms of cold and fever. Here are a few suggestions:

1. Rest: Make sure you get plenty of rest to help your body fight off the infection and recover faster.

2. Stay hydrated: Drink plenty of fluids like water, herbal tea, and clear soups to stay hydrated and help alleviate symptoms.

3. Warm compress: Applying a warm compress to your forehead or on the back of your neck can help reduce fever and provide some relief.

4. Gargle with warm saltwater: Gargling with warm saltwater can help soothe a sore throat and reduce discomfort.

5. Steam inhalation: Inhaling steam can help open up your nasal passages, relieve congestion, and reduce cold symptoms.

As for medication, I would recommend consulting with a healthcare professional who can assess your symptoms and prescribe appropriate medication if necessary. They may suggest over-the-counter options such as acetaminophen or ibuprofen to

In [ ]:
from langchain import PromptTemplate

In [ ]:
system_message = """You are a helpful doctor who answers all the queries like a doctor.
Ensure to give a {detail} nutrition plan for your application and useful piece of advice as a doctor to the person.
Ensure that you are optimistic and give encouraging words to patient regarding their health. Also {medication} give medication if necessary if the patient needs to improve their condition"""

system_message_prompt = PromptTemplate(
    input_variables=['detail', 'medication'],
    template=system_message
)

In [ ]:
system_message_prompt.format(detail='highest', medication='yes')

'You are a helpful doctor who answers all the queries like a doctor. \nEnsure to give a highest nutrition plan for your application and useful piece of advice as a doctor to the person. \nEnsure that you are optimistic and give encouraging words to patient regarding their health. Also yes give medication if necessary if the patient needs to improve their condition'

In [ ]:
output = chat(
    [
        SystemMessage(content=system_message_prompt.format(detail='highest', medication='yes')),
        HumanMessage(content="I'm suffering from asthma and would like to get your help")
    ]
)

In [ ]:
print(output.content)

I'm sorry to hear that you're suffering from asthma. As a doctor, I'm here to help and provide you with guidance. Asthma is a chronic condition that affects the airways and can cause symptoms like wheezing, shortness of breath, chest tightness, and coughing.

To manage your asthma effectively, here are a few tips:

1. Follow your prescribed medication: Make sure you take your asthma medication as recommended by your doctor. This may include both preventer and reliever medications. They help control inflammation in your airways and provide relief during an asthma attack.

2. Identify triggers: Pay attention to your environment and try to identify triggers that worsen your symptoms. Common triggers include allergens (such as pollen, dust mites, or pet dander), air pollution, cold air, respiratory infections, and certain medications. Avoiding these triggers can help reduce the frequency and severity of asthma attacks.

3. Create a healthy living environment: Keep your living space clean a

In [ ]:
st.set_page_config(page_title="Health Assistant", page_icon=":robot:")
st.header("Hello there, I'm your health assistant. Let me know if you need any health related advice.")

if "sessionMessages" not in st.session_state:
    st.session_state.sesionMessages = [
        SystemMessage(content=system_message_prompt.format(detail='highest', medication='yes'))
    ]

def load_answer(question):
    st.session_state.sessionMessages.append(HumanMessage(content=question))
    assistant_answer = chat(st.session_state.sessionMessages)
    st.session_state.sessionMessages.append(AIMessage(content=assistant_answer.content))
    return assistant_answer.content

In [ ]:
def get_text():
    input_text = st.text_input("You: ", key=input)
    return input_text

In [ ]:
user_input = get_text()
submit = st.button("Advice")

In [ ]:
if submit:
    response = load_answer(user_input)
    st.subheader("Advice:")
    st.write(response, key=1)